In [1]:
import os
HOME = os.getcwd()
print(HOME)

In [2]:
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.94 🚀 Python-3.9.7 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Setup complete ✅ (20 CPUs, 62.5 GB RAM, 200.9/227.6 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="T3E2M8jPKMaklrg9Ouoq")
project = rf.workspace("runaway").project("cats-bfpvf")
version = project.version(14)
dataset = version.download("yolov8-obb", location="./seg-14")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./seg-14 in yolov8-obb:: 100%|█████████████████| 1662/1662 [00:00<00:00, 13175.77it/s]


In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="T3E2M8jPKMaklrg9Ouoq")
project = rf.workspace("runaway").project("cats-bfpvf")
version = project.version(13)
dataset = version.download("yolov8-obb", location="./delete")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./delete in yolov8-obb:: 100%|█████████████████| 1264/1264 [00:00<00:00, 10344.34it/s]


In [4]:

from roboflow import Roboflow
rf = Roboflow(api_key="T3E2M8jPKMaklrg9Ouoq")
project = rf.workspace("runaway").project("cats-bfpvf")
version = project.version(13)
dataset = version.download("coco", location="./coco-bounding")

In [7]:
from ultralytics.data.converter import convert_coco

convert_coco(labels_dir="./coco-bounding/train")

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="T3E2M8jPKMaklrg9Ouoq")
project = rf.workspace("runaway").project("cats-bfpvf")
version = project.version(13)
dataset = version.download("yolov8", location="./bounding-box")
                

In [12]:
!mkdir {HOME}/roboflow_datasets
%cd {HOME}/roboflow_datasets

from roboflow import Roboflow
rf = Roboflow(api_key="T3E2M8jPKMaklrg9Ouoq")
project = rf.workspace("runaway").project("cats-bfpvf")
version = project.version(13)
dataset = version.download("yolov8-obb")
                

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolov8s-seg.pt data={dataset.location}/data.yaml epochs=200 imgsz=500

In [1]:
!yolo task=segment mode=train model=yolov8s-seg.pt data=./cropped_roboflow5-224/data.yaml epochs=400 imgsz=224

In [5]:
!ls runs/segment/train32

In [10]:
%cd {HOME}
Image(filename=f'runs/segment/train32/confusion_matrix.png', width=600)

In [5]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train35/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train23/val_batch0_pred.jpg', width=600)

In [1]:
!ls

bounding-box		 runaway-seg-11
bounding_box_224	 runs
cats-13			 SampleSubmission.csv
coco-bounding		 seg-converted-bb
coco_converted		 seg-converted-bb-full-augmentation
coco_converted_224	 seg-data
cropped_roboflow	 seg-data2
cropped_roboflow-12-200  separated_locations_224
cropped_roboflow2	 submission.csv
cropped_roboflow-256	 test-data-masked
cropped_roboflow3-224	 test_images
cropped_roboflow4-224	 test.png
cropped_roboflow5-224	 venv
datasets		 yolov10x.pt
imaug			 yolov8n.pt
results			 yolov8s-seg.pt
roboflow_datasets	 yolov8x.pt
runaway-seg-10		 yolov8x-seg.pt


## Validate Custom Model

In [ ]:
!yolo task=segment mode=val model=./runs/segment/train22/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train23/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train24/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train25/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train29/weights/best.pt data=./cropped_roboflow2/data.yaml

In [2]:
!yolo task=segment mode=val model=./runs/segment/train35/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train36/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo task=segment mode=val model=./runs/segment/train35/weights/best.pt data=./cropped_roboflow5-224/data.yaml
!yolo task=segment mode=val model=./runs/segment/train36/weights/best.pt data=./cropped_roboflow5-224/data.yaml
!yolo detect val model=./runs/detect/train20/weights/best.pt data=./cropped_roboflow2/data.yaml
!yolo detect val model=./runs/detect/train20/weights/best.pt data=./cropped_roboflow5-224/data.yaml

Ultralytics YOLOv8.2.94 🚀 Python-3.9.7 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs
val: Scanning /home/p.kuznetsov/runaway/cropped_roboflow2/valid/labels.cache... 
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all         55         55      0.997      0.945      0.987      0.847      0.937      0.927      0.931      0.603
Speed: 1.1ms preprocess, 7.2ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to runs/segment/val8
💡 Learn more at https://docs.ultralytics.com/modes/val
Ultralytics YOLOv8.2.94 🚀 Python-3.9.7 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs
val: Scanning /home/p.kuznetsov/runaway/cropped_roboflow2/valid/labels.cache... 
                 Class     Images  Instances      Box(P          R  

In [3]:
!yolo task=segment mode=val model=./runs/segment/train35/weights/best.pt data=./seg-converted-bb-full-augmentation/data.yaml
!yolo task=segment mode=val model=./runs/segment/train36/weights/best.pt data=./seg-converted-bb-full-augmentation/data.yaml
!yolo detect val model=./runs/detect/train20/weights/best.pt data=./seg-converted-bb-full-augmentation/data.yaml

Ultralytics YOLOv8.2.94 🚀 Python-3.9.7 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs
val: Scanning /home/p.kuznetsov/runaway/seg-converted-bb-full-augmentation/valid
                 Class     Images  Instances      Box(P          R      mAP50  m
Traceback (most recent call last):
  File "/home/p.kuznetsov/runaway/venv/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/home/p.kuznetsov/runaway/venv/lib/python3.9/site-packages/ultralytics/cfg/__init__.py", line 830, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/home/p.kuznetsov/runaway/venv/lib/python3.9/site-packages/ultralytics/engine/model.py", line 648, in val
    validator(model=self.model)
  File "/home/p.kuznetsov/runaway/venv/lib/python3.9/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/home/p.kuznets

## Inference with Custom Model

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/segment/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, height=600))
      print("\n")

## Save & Deploy model

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload and your model weights to Roboflow Deploy for autolabeling, autoscaling inference, and using later.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) supports uploading YOLOv8 weights.

Run this cell to save your model weights:

In [9]:
project.version(dataset.version).deploy(model_type="yolov8-seg", model_path=f"./runs/segment/train37/")

Dependency ultralytics==8.0.196 is required but found version=8.2.94, to fix: `pip install ultralytics==8.0.196`
View the status of your deployment at: https://app.roboflow.com/runaway/cats-bfpvf/13
Share your model with the world at: https://universe.roboflow.com/runaway/cats-bfpvf/model/13
